In [1]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

c:\users\caleb\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18071281604635251294
]


In [3]:
import tensorflow as tf
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

c:\users\caleb\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[[22. 28.]
 [49. 64.]]


In [1]:
import lyricwikia
import pandas as pd

df = pd.read_csv("pop_playlist_project.csv")
df.columns = ['Spotify_URI','TrackName','ArtistName','AlbumName','DiscNum','TrackNum','TrackDur','AddedBy','AddedAt']
lyrics = ""

for i in range(int(len(df)/30)):
    try:
        lyrics_temp = lyricwikia.get_lyrics(df.ArtistName[i], df.TrackName[i])
        lyrics=lyrics+lyrics_temp
    except:
        continue
#print(lyrics)

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

c:\users\caleb\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
raw_text = lyrics.lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ",n_vocab)

Total Characters:  40151
Total Vocab:  42


In [4]:
seq_length = len(df)
dataX = []
dataY = []
for i in range(0,n_chars - seq_length,1):
    seq_in = raw_text[i:i +seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ",n_patterns)

Total Patterns:  39302


In [5]:
X = np.reshape(dataX,(n_patterns,seq_length,1))
X = X/float(n_vocab)
y=np_utils.to_categorical(dataY)

In [14]:
print(y.shape[1])

42


In [12]:
model = Sequential()
#256
model.add(LSTM(15,input_shape=(X.shape[1],X.shape[2]),return_sequences = True))
model.add(Dropout(0.2))
#256
model.add(LSTM(15))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [13]:
model.fit(X,y,batch_size=15)
#epochs=10,batch_size=128

Epoch 1/1
39302/39302 [==============================] - 1637s 42ms/step - loss: 3.0615


In [16]:
int_to_char = dict((i,c) for i,c in enumerate(chars))
#print(int_to_char)
initial_text=("My lyrics for my random pop song").lower()
pattern=[char_to_int[char] for char in initial_text]
pattern=pattern[0:seq_length]

text_gen=""

for i in range(1000):
    x = np.reshape(pattern,(1,32,1))
    x = x/float(n_vocab)
    prediction = model.predict(x,verbose=0)
    prediction = prediction[0]
    index = np.random.choice(len(prediction),p=prediction)
    result = int_to_char[index]
    text_gen = text_gen+result
    pattern.append(index)
    pattern=pattern[1:len(pattern)]
    
print(text_gen)

ValueError: Error when checking : expected lstm_5_input to have shape (849, 1) but got array with shape (32, 1)